# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/00_introduction/00_exercices_solutions.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '00_exercices_solutions.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 00 - Solutions des Exercices

Ce notebook contient les solutions détaillées des exercices du Chapitre 00.

**Remarque** : Essayez d'abord de résoudre les exercices par vous-même avant de consulter les solutions !

---

## Setup Initial

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_wine, load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
np.random.seed(42)

print("✓ Bibliothèques importées")

---

## Solution Exercice 1 : Analyse Exploratoire (EDA)

### 1.1 Chargement et Exploration

In [ ]:
# Chargement du dataset Wine
wine = load_wine()

# Création du DataFrame
df_wine = pd.DataFrame(
    data=wine.data,  # type: ignore
    columns=wine.feature_names  # type: ignore
)
df_wine['target'] = wine.target  # type: ignore
df_wine['target_name'] = df_wine['target'].map(
    {i: name for i, name in enumerate(wine.target_names)}  # type: ignore
)

print("Dataset Wine chargé")
display(df_wine.head(10))

In [ ]:
# Réponses aux questions

# 1. Nombre d'échantillons et features
n_samples, n_features = wine.data.shape  # type: ignore
print(f"1. Nombre d'échantillons : {n_samples}")
print(f"   Nombre de features : {n_features}")

# 2. Classes cibles
print(f"\n2. Classes cibles : {wine.target_names}")  # type: ignore
print(f"   Nombre de classes : {len(wine.target_names)}")  # type: ignore

# 3. Valeurs manquantes
print(f"\n3. Valeurs manquantes :")
print(df_wine.isnull().sum())
print("   → Aucune valeur manquante")

### 1.2 Statistiques Descriptives

In [ ]:
# Statistiques descriptives
display(df_wine.describe())

print("\nObservations :")
print("- Les features ont des échelles très différentes (ex: proline ~746, malic_acid ~2.3)")
print("- La standardisation sera importante pour certains algorithmes")

### 1.3 Distribution des Classes

In [ ]:
# Distribution des classes
print("Distribution des classes :")
print(df_wine['target_name'].value_counts())

# Visualisation
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Countplot
sns.countplot(data=df_wine, x='target_name', hue='target_name', ax=ax[0], palette='Set2', legend=False)
ax[0].set_title('Distribution des Classes', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Classe')
ax[0].set_ylabel('Nombre d\'échantillons')

# Pie chart
counts = df_wine['target_name'].value_counts()
ax[1].pie(
    counts,
    labels=counts.index,
    autopct='%1.1f%%',
    colors=sns.color_palette('Set2'),
    startangle=90
)
ax[1].set_title('Proportion des Classes', fontsize=14, fontweight='bold')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

print("\nRéponse : Le dataset est légèrement déséquilibré")
print("class_0: 59 échantillons (33%), class_1: 71 (40%), class_2: 48 (27%)")

### 1.4 Matrice de Corrélation

In [ ]:
# Matrice de corrélation
corr_matrix = df_wine[list(wine.feature_names)].corr()  # type: ignore

# Visualisation
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_matrix,
    annot=True,
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},
    fmt='.2f',
    annot_kws={'size': 8}
)
plt.title('Matrice de Corrélation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Identifier les corrélations les plus fortes
print("\nPaires de features les plus corrélées :")
corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_pairs.append((
            corr_matrix.columns[i],
            corr_matrix.columns[j],
            abs(corr_matrix.iloc[i, j])
        ))

corr_pairs_sorted = sorted(corr_pairs, key=lambda x: x[2], reverse=True)
for feat1, feat2, corr in corr_pairs_sorted[:5]:
    print(f"  {feat1[:20]:20s} ↔ {feat2[:20]:20s} : {corr:.3f}")

### 1.5 Pairplot

In [ ]:
# Pairplot pour les 4 premières features
features_to_plot = wine.feature_names[:4]  # type: ignore
df_subset = df_wine[list(features_to_plot) + ['target_name']]

sns.pairplot(
    df_subset,
    hue='target_name',
    palette='Set2',
    diag_kind='kde',
    markers=['o', 's', 'D'],
    plot_kws={'alpha': 0.6}
)
plt.suptitle('Pairplot - 4 Premières Features', y=1.02, fontsize=14, fontweight='bold')
plt.show()

---

## Solution Exercice 2 : Pipeline ML Complet

### 2.1 Chargement des Données

In [ ]:
# Chargement du dataset Breast Cancer
cancer = load_breast_cancer()
X = cancer.data  # type: ignore
y = cancer.target  # type: ignore

print(f"Nombre d'échantillons : {X.shape[0]}")
print(f"Nombre de features : {X.shape[1]}")
print(f"Classes : {cancer.target_names}")  # type: ignore
print(f"Distribution : {np.bincount(y)} (0=malignant, 1=benign)")

### 2.2 Split Train/Test

In [ ]:
# Split 70/30 avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"Train set : {X_train.shape[0]} échantillons ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set  : {X_test.shape[0]} échantillons ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\nDistribution train : {np.bincount(y_train)}")
print(f"Distribution test  : {np.bincount(y_test)}")

### 2.3 Standardisation

In [ ]:
# Standardisation (fit sur train uniquement)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Standardisation effectuée")
print(f"Moyennes (train) : {X_train_scaled.mean(axis=0)[:5].round(2)}")
print(f"Écart-types (train) : {X_train_scaled.std(axis=0)[:5].round(2)}")

### 2.4 Entraînement de Modèles

In [ ]:
# Dictionnaire de modèles
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', random_state=42, probability=True)
}

results = {}

print("Entraînement des modèles...\n")

for name, model in models.items():
    print(f"⏳ {name}...")
    
    # Entraînement
    if name in ['Logistic Regression', 'SVM']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    
    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    
    # Stockage
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'y_pred': y_pred
    }
    
    print(f"   Accuracy : {accuracy:.4f}")
    print(f"   CV Score : {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})\n")

print("✅ Entraînement terminé")

### 2.5 Évaluation et Comparaison

In [ ]:
# DataFrame de comparaison
comparison = pd.DataFrame({
    'Modèle': list(results.keys()),
    'Accuracy': [r['accuracy'] for r in results.values()],
    'CV Mean': [r['cv_mean'] for r in results.values()],
    'CV Std': [r['cv_std'] for r in results.values()]
}).sort_values('Accuracy', ascending=False)

print("Comparaison des performances :")
display(comparison)

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))
x_pos = np.arange(len(comparison))

ax.barh(
    x_pos,
    comparison['Accuracy'],
    color=sns.color_palette('viridis', len(comparison)),
    edgecolor='black'
)
ax.set_yticks(x_pos)
ax.set_yticklabels(comparison['Modèle'])
ax.set_xlabel('Accuracy', fontsize=12)
ax.set_title('Comparaison des Performances', fontsize=14, fontweight='bold')
ax.set_xlim([0.9, 1.0])
ax.grid(axis='x', alpha=0.3)

# Annotations
for i, v in enumerate(comparison['Accuracy']):
    ax.text(v + 0.005, i, f"{v:.4f}", va='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 2.6 Évaluation Détaillée du Meilleur Modèle

In [ ]:
# Sélection du meilleur modèle
best_model_name = comparison.iloc[0]['Modèle']
best_model = results[best_model_name]['model']
best_y_pred = results[best_model_name]['y_pred']

print(f"🏆 Meilleur modèle : {best_model_name}\n")

# Classification report
print("Classification Report :")
print(classification_report(
    y_test,
    best_y_pred,
    target_names=cancer.target_names  # type: ignore
))

# Confusion matrix
cm = confusion_matrix(y_test, best_y_pred)

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Matrice absolue
disp1 = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=cancer.target_names  # type: ignore
)
disp1.plot(ax=ax[0], cmap='Blues', values_format='d')
ax[0].set_title('Matrice de Confusion', fontsize=12, fontweight='bold')

# Matrice normalisée
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
disp2 = ConfusionMatrixDisplay(
    confusion_matrix=cm_norm,
    display_labels=cancer.target_names  # type: ignore
)
disp2.plot(ax=ax[1], cmap='Blues', values_format='.2%')
ax[1].set_title('Matrice de Confusion (Normalisée)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

---

## Solution Exercice 3 : Détection d'Overfitting

### 3.1 Création d'un Dataset avec Peu d'Échantillons

In [ ]:
# Dataset Wine avec seulement 30 échantillons
wine = load_wine()

# Sélection aléatoire de 30 indices
indices = np.random.choice(len(wine.data), size=30, replace=False)  # type: ignore

X_small = wine.data[indices]  # type: ignore
y_small = wine.target[indices]  # type: ignore

# Split 20/10
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    X_small, y_small,
    test_size=10,
    random_state=42,
    stratify=y_small
)

print(f"Train : {len(X_train_small)} échantillons")
print(f"Test  : {len(X_test_small)} échantillons")

### 3.2 Entraînement d'un Modèle Complexe

In [ ]:
# Decision Tree sans limitation de profondeur
tree_overfit = DecisionTreeClassifier(max_depth=None, random_state=42)
tree_overfit.fit(X_train_small, y_train_small)

# Accuracy sur train et test
train_accuracy = tree_overfit.score(X_train_small, y_train_small)
test_accuracy = tree_overfit.score(X_test_small, y_test_small)

print(f"Train Accuracy : {train_accuracy:.4f}")
print(f"Test Accuracy  : {test_accuracy:.4f}")

print("\n💡 Analyse :")
if train_accuracy > test_accuracy + 0.1:
    print("   ⚠️ OVERFITTING détecté !")
    print("   Le modèle a une performance parfaite sur le train (1.00)")
    print("   mais beaucoup plus faible sur le test.")
    print("   → Le modèle a mémorisé les données d'entraînement.")
else:
    print("   ✓ Pas d'overfitting majeur détecté")

### 3.3 Régularisation pour Réduire l'Overfitting

In [ ]:
# Test de différentes profondeurs
max_depths = [2, 3, 4, 5, 6, 7, 8, None]
train_scores = []
test_scores = []

for depth in max_depths:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree.fit(X_train_small, y_train_small)
    
    train_scores.append(tree.score(X_train_small, y_train_small))
    test_scores.append(tree.score(X_test_small, y_test_small))

# Visualisation
plt.figure(figsize=(12, 6))

# Conversion de None en "No limit" pour l'affichage
depths_labels = [str(d) if d is not None else "No limit" for d in max_depths]
x_pos = np.arange(len(max_depths))

plt.plot(x_pos, train_scores, 'o-', linewidth=2, markersize=8, label='Train', color='blue')
plt.plot(x_pos, test_scores, 's-', linewidth=2, markersize=8, label='Test', color='red')

plt.xticks(x_pos, depths_labels)
plt.xlabel('max_depth', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Train vs Test Accuracy en fonction de max_depth', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.ylim([0, 1.05])

# Zone d'overfitting
plt.axvspan(5.5, 7.5, alpha=0.2, color='red', label='Zone d\'overfitting')

plt.tight_layout()
plt.show()

# Meilleure profondeur
best_depth_idx = np.argmax(test_scores)
best_depth = max_depths[best_depth_idx]
print(f"\n✅ Meilleure profondeur : {best_depth}")
print(f"   Test Accuracy : {test_scores[best_depth_idx]:.4f}")

---

## Solution Exercice 4 : Feature Engineering

### 4.1 Création de Nouvelles Features

In [ ]:
# Dataset Iris
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)  # type: ignore
df_iris['target'] = iris.target  # type: ignore

# Création de nouvelles features
df_iris['sepal_ratio'] = df_iris['sepal length (cm)'] / df_iris['sepal width (cm)']
df_iris['petal_ratio'] = df_iris['petal length (cm)'] / df_iris['petal width (cm)']
df_iris['petal_area'] = df_iris['petal length (cm)'] * df_iris['petal width (cm)']

print("Nouvelles features créées :")
display(df_iris.head(10))

# Visualisation des nouvelles features
fig, ax = plt.subplots(1, 3, figsize=(16, 5))

for i, (col, axis) in enumerate(zip(['sepal_ratio', 'petal_ratio', 'petal_area'], ax)):
    for target in range(3):
        subset = df_iris[df_iris['target'] == target]
        axis.hist(
            subset[col],
            alpha=0.6,
            label=iris.target_names[target],  # type: ignore
            bins=15,
            edgecolor='black'
        )
    axis.set_xlabel(col, fontsize=11)
    axis.set_ylabel('Fréquence', fontsize=11)
    axis.set_title(f'Distribution : {col}', fontsize=12, fontweight='bold')
    axis.legend()
    axis.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 4.2 Comparaison Avant/Après Feature Engineering

In [ ]:
# Modèle sans nouvelles features
X_original = iris.data  # type: ignore
y = iris.target  # type: ignore

model_original = RandomForestClassifier(n_estimators=100, random_state=42)
cv_original = cross_val_score(model_original, X_original, y, cv=5)

print("Sans feature engineering :")
print(f"  CV Mean : {cv_original.mean():.4f}")
print(f"  CV Std  : {cv_original.std():.4f}")

# Modèle avec nouvelles features
feature_cols = [col for col in df_iris.columns if col != 'target']
X_engineered = df_iris[feature_cols].values

model_engineered = RandomForestClassifier(n_estimators=100, random_state=42)
cv_engineered = cross_val_score(model_engineered, X_engineered, y, cv=5)

print("\nAvec feature engineering :")
print(f"  CV Mean : {cv_engineered.mean():.4f}")
print(f"  CV Std  : {cv_engineered.std():.4f}")

# Comparaison
improvement = cv_engineered.mean() - cv_original.mean()
print(f"\n✅ Amélioration : {improvement:.4f} ({improvement/cv_original.mean()*100:.2f}%)")

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))

bp = ax.boxplot(
    [cv_original, cv_engineered],
    labels=['Sans FE', 'Avec FE'],
    patch_artist=True,
    boxprops=dict(facecolor='lightblue', edgecolor='black'),
    medianprops=dict(color='red', linewidth=2)
)

ax.set_ylabel('CV Accuracy', fontsize=12)
ax.set_title('Impact du Feature Engineering', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## Solution Exercice 5 : Interprétation des Résultats

### 5.1 Analyse d'une Matrice de Confusion

In [ ]:
# Matrice de confusion fictive
cm = np.array([
    [85, 15],  # Classe 0 (non-spam)
    [10, 90]   # Classe 1 (spam)
])

# Calcul des composantes
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives
TP = cm[1, 1]  # True Positives

print("Composantes de la matrice de confusion :")
print(f"  TN (True Negatives)  : {TN}")
print(f"  FP (False Positives) : {FP}")
print(f"  FN (False Negatives) : {FN}")
print(f"  TP (True Positives)  : {TP}")

# Calcul des métriques
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * (precision * recall) / (precision + recall)

print("\nMétriques :")
print(f"  Accuracy  : {accuracy:.4f}")
print(f"  Precision : {precision:.4f}")
print(f"  Recall    : {recall:.4f}")
print(f"  F1-Score  : {f1:.4f}")

print("\n💡 Interprétation dans le contexte du spam :")
print("\n  RECALL (Sensibilité) est la métrique la plus importante car :")
print("  - On veut détecter un maximum de spams (minimiser les FN)")
print("  - Un spam non détecté (FN) peut contenir malware/phishing")
print("  - Un email légitime classé spam (FP) est moins grave")
print("    (l'utilisateur peut vérifier son dossier spam)")
print(f"\n  Recall actuel : {recall:.4f} (90% des spams détectés)")
print(f"  → 10 spams passent à travers (10 FN)")

---

## Solution Exercice 6 : Questions de Réflexion

### 6.1 Overfitting vs Underfitting

**Réponse** :

**Overfitting (Surapprentissage)** :
- Le modèle apprend "par cœur" les données d'entraînement, incluant le bruit
- Performance excellente sur le train set, mais mauvaise sur le test set
- Le modèle est trop complexe par rapport aux données disponibles
- **Exemple** : Un arbre de décision avec une profondeur illimitée sur un petit dataset mémorise chaque échantillon (accuracy train = 100%, test = 60%)

**Underfitting (Sous-apprentissage)** :
- Le modèle est trop simple pour capturer les patterns des données
- Performance faible sur le train set ET le test set
- Le modèle manque de capacité d'apprentissage
- **Exemple** : Utiliser une régression linéaire pour modéliser une relation non-linéaire (ex: relation quadratique). Le modèle ne peut pas capturer la courbe.

**Solutions** :
- Overfitting : Régularisation (L1/L2), réduction de complexité, plus de données, dropout
- Underfitting : Modèle plus complexe, plus de features, moins de régularisation

### 6.2 Choix de Modèle

**Réponse** :

Pour prédire le prix d'une maison, je recommanderais :

**1. Gradient Boosting (XGBoost, LightGBM, CatBoost)** - Choix principal
- **Pourquoi** :
  - Excellent sur les données tabulaires
  - Gère bien les relations non-linéaires
  - Robuste aux outliers
  - Gère automatiquement les features catégorielles (CatBoost)
  - Performances state-of-the-art sur ce type de problème
- **Inconvénients** :
  - Moins interprétable qu'une régression linéaire
  - Nécessite un tuning d'hyperparamètres

**2. Random Forest Regressor** - Alternative solide
- **Pourquoi** :
  - Robuste, peu de risque d'overfitting
  - Fournit des feature importances
  - Moins de tuning nécessaire
- **Inconvénients** :
  - Performances légèrement inférieures au Gradient Boosting

**3. Ridge Regression** - Pour la baseline et l'interprétabilité
- **Pourquoi** :
  - Très interprétable (coefficients)
  - Rapide à entraîner
  - Bon pour comprendre l'impact de chaque feature
- **Inconvénients** :
  - Assume une relation linéaire
  - Performances limitées si la relation est complexe

**Approche recommandée** :
1. Commencer par Ridge (baseline interprétable)
2. Essayer Random Forest
3. Optimiser avec Gradient Boosting (meilleure performance)
4. Comparer avec validation croisée
5. Feature engineering : ratios, interactions, transformations log

### 6.3 Éthique en ML

**Réponse** :

**Biais potentiels dans un système de tri de CV** :

1. **Biais de genre** :
   - Historique : Si l'entreprise a historiquement embauché plus d'hommes, le modèle peut apprendre ce biais
   - Indices indirects : Prénoms, loisirs genrés ("football" vs "yoga")

2. **Biais racial/ethnique** :
   - Noms de famille, établissements d'enseignement dans certains quartiers
   - Langues parlées

3. **Biais d'âge** :
   - Année de diplôme, nombre d'années d'expérience
   - Technologies anciennes vs récentes

4. **Biais socio-économique** :
   - Université prestigieuse vs moins connue
   - Expériences à l'étranger (coût prohibitif pour certains)

**Solutions pour atténuer ces biais** :

1. **Audit des données** :
   - Analyser la distribution des candidats embauchés par démographie
   - Identifier les déséquilibres historiques

2. **Anonymisation** :
   - Retirer nom, prénom, âge, photo
   - Masquer les années (remplacer par "3-5 ans d'expérience")

3. **Feature selection prudente** :
   - Exclure les features sensibles (genre, ethnie, âge)
   - Éviter les proxies (codes postaux, établissements)

4. **Données d'entraînement équilibrées** :
   - Rééquilibrage (oversampling/undersampling)
   - Utiliser des données synthétiques pour les groupes sous-représentés

5. **Fairness constraints** :
   - Imposer des contraintes d'équité pendant l'entraînement
   - Vérifier le taux de sélection par groupe démographique

6. **Human-in-the-loop** :
   - Le modèle assiste, ne décide pas seul
   - Revue humaine des candidatures recommandées
   - Diversité dans l'équipe de recrutement

7. **Monitoring continu** :
   - Suivre les taux d'acceptation par démographie
   - Audits réguliers par une équipe indépendante
   - Feedback des candidats rejetés

8. **Transparence** :
   - Documenter le fonctionnement du système
   - Expliquer aux candidats comment les décisions sont prises
   - Droit de contestation

**Principe fondamental** : Le ML doit augmenter l'humain, pas le remplacer, surtout pour des décisions impactant la vie des personnes.

---

## Conclusion

Félicitations pour avoir complété ces exercices !

Vous avez maintenant une bonne compréhension des concepts fondamentaux du Machine Learning :
- Pipeline ML complet
- EDA et visualisation
- Détection et prévention de l'overfitting
- Validation croisée
- Feature engineering
- Interprétation des métriques
- Considérations éthiques

**Prochaines étapes** :
1. Pratiquer sur d'autres datasets (Kaggle)
2. Approfondir les mathématiques (Chapitre 01)
3. Explorer les algorithmes en détail (Chapitres 02-10)

---